In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
set(stopwords.words('english'))

In [74]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import adjusted_rand_score

In [11]:
nlp = spacy.load("en_core_web_lg")

In [13]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [46]:
url = "https://raw.githubusercontent.com/patilpranjal/Auto-Grader-for-short-answer-Question-using-Siamese-Neural-Network/master/Dataset.csv"
df = pd.read_csv(
    url,
    index_col= 0
)

# Basic Dataset Description

In [47]:
df.head()

,studentAnswer,referenceAnswer,ref_1,ref_2,accuracy
0,By letting it sit in a dish for a day.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect
1,Let the water evaporate and the salt is left b...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,correct
2,The water evaporated and left salt crystals.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,correct
3,I saw a pinkish grayish color that was blockin...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect
4,You have to slowly tip the vial for only the w...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,incorrect


In [48]:
df.referenceAnswer.value_counts().size
# 135 Unique Questions

135

In [49]:
df.referenceAnswer.value_counts().mean()
# 36.5 student responses

36.48148148148148

In [50]:
df["accuracy"] = df.accuracy.apply(lambda x: 1 if x == "correct" else 0)

,studentAnswer,referenceAnswer,ref_1,ref_2,accuracy
0,By letting it sit in a dish for a day.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0
1,Let the water evaporate and the salt is left b...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1
2,The water evaporated and left salt crystals.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1
3,I saw a pinkish grayish color that was blockin...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0
4,You have to slowly tip the vial for only the w...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0
5,"By pouring the water and salt into the thing, ...","The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1
6,By slowly pouring it in a tray.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0
7,The water evaporated so there was only salt left.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1
8,We put the water with the salt in it and put i...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1
9,I separated the salt from the water because th...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0


In [65]:
unique = pd.DataFrame(
    df.referenceAnswer.value_counts(sort = False).index,
    columns = ["referenceAnswer"]
).reset_index()

In [71]:
dataset = pd.merge(
    left = df,
    right = unique,
    on = "referenceAnswer",
    how = "left"
)

In [73]:
dataset[dataset['index'] == 101]

,studentAnswer,referenceAnswer,ref_1,ref_2,accuracy,index
0,By letting it sit in a dish for a day.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0,101
1,Let the water evaporate and the salt is left b...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1,101
2,The water evaporated and left salt crystals.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1,101
3,I saw a pinkish grayish color that was blockin...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0,101
4,You have to slowly tip the vial for only the w...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0,101
5,"By pouring the water and salt into the thing, ...","The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1,101
6,By slowly pouring it in a tray.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0,101
7,The water evaporated so there was only salt left.,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1,101
8,We put the water with the salt in it and put i...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,1,101
9,I separated the salt from the water because th...,"The water was evaporated, leaving the salt.",We evaporated the salt from the water because ...,I put some water in a dish and let the water e...,0,101


# Removing Stop Words

In [23]:
from spacy.lang.en.stop_words import STOP_WORDS
def remove_stopwords(doc):
    my_doc = nlp(doc)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    
    filtered_sentence =[] 
    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    return " ".join(filtered_sentence)

In [43]:
# Teachers Answer
answer = "The water was evaporated, leaving the salt."
remove_stopwords(answer)

'water evaporated , leaving salt .'

In [40]:
# Incorrect Answer
remove_stopwords('By letting it sit in a dish for a day.')

'letting sit dish day .'

In [42]:
# Correct Answer
remove_stopwords('Let the water evaporate and the salt is left behind.')

'Let water evaporate salt left .'

In [44]:
df["filtered_answer"] = df.studentAnswer.apply(remove_stopwords)

# Quick Clustering with the 101 Set

In [ ]:
documents = df[df['group'] == 12].studentAnswer.values
label = df[df['group'] == 12].accuracy.values

In [78]:
docs = dataset[dataset['index'] == 101].studentAnswer.values

In [79]:
vectorizer = TfidfVectorizer(stop_words=STOP_WORDS)
X = vectorizer.fit_transform(docs)

c:\users\imcna\.virtualenvs\answer_clustering-dp5vzxyv\lib\site-packages\sklearn\feature_extraction\text.py:382: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [80]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
y_pred = model.predict(X)

In [81]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind]),
    print()

Top terms per cluster:
Cluster 0:
 water
 evaporate
 dish
 let
 poured

Cluster 1:
 salt
 water
 evaporated
 crystals
 left

